In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from myfunc import timer
from myfunc import DirMan
import config
import matplotlib

pd.set_option('display.max_columns', None)
font = {'family': 'Times New Roman'}
matplotlib.rc('font', **font)

# configuration
# resolution = "0p1"
resolution = "500"
region = [-180,180,-60,90]
data_path = f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/'
post_data_path = '/tera04/zhwei/xionghui/bedrock/'
shp_path = '/tera04/zhwei/xionghui/bedrock/Shp/'
fig_path = f'/home/xuxh22/stu01/Bedrock/fig/{resolution}/'
path = '/home/xuxh22/stu01/Bedrock/'
if resolution == "0p1":
    size = 0.1
elif resolution == "500":
    size = 0.0005

print(data_path)

df = pd.read_csv(f'/tera04/zhwei/xionghui/bedrock/run/{resolution}/csv/Global_DF.csv')
df

/tera04/zhwei/xionghui/bedrock/run/500/


,Unnamed: 0,lat,lon,Dbedrock_Frequency,Sbedrock,Area,Koppen,IGBP
0,708161369,-55.847915,-67.627077,1.0,0.236460,120515.621801,29,1
1,708161370,-55.847915,-67.622911,1.0,0.279453,120515.621801,29,1
2,708161371,-55.847915,-67.618744,1.0,0.322446,120515.621801,29,1
3,708161372,-55.847915,-67.614577,1.0,0.365440,120515.621801,29,4
4,708161373,-55.847915,-67.610411,1.0,0.408433,120515.621801,29,4
...,...,...,...,...,...,...,...,...
157069426,3586500782,82.960426,-26.739575,1.0,39.301605,26309.225697,29,1
157069427,3586500783,82.960426,-26.735409,1.0,39.884820,26309.225697,29,1
157069428,3586500784,82.960426,-26.731242,1.0,40.468037,26309.225697,29,1
157069429,3586500785,82.960426,-26.727075,1.0,41.051250,26309.225697,29,1


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

df1= df.copy()
df1= df1[df1['Koppen'] > 0]
df1= df1[df1['Koppen'] < 29]
df1 = df1[df1['IGBP'] < 10]
df1 = df1[df1['IGBP'] > 0]

df1['Koppen_Together'] = df1['Koppen'].replace(to_replace=[5, 7, 9, 10, 12, 13, 15, 16, 18, 19, 20, 22, 23, 24, 26, 27, 28], value=[4, 6, 8, 8, 11, 11, 14, 14, 17, 17, 17, 21, 21, 21, 25, 25, 25])

KoppenList = ['Tropical Rainforest (Af)','Tropical Monsoon (Am)','Tropical Savanna (Aw)',
              'Desert (BWh)','Arid (BWk)',
              'Semi Arid (BSh)','Semi Arid (BSk)',
              'Mediterranean (Csa)','Mediterranean (Csb)','Mediterranean (Csc)',
              'Humid Subtropical (Cwa)','Oceanic (Cwb)','Ocanic (Cwc)',
              'Humid Subtropical (Cfa)','Oceanic (Cfb)','Oceanic (Cfc)',
              'Humid Continental (Dsa)','Humid Continental (Dsb)','Subarctic (Dsc)','Subarctic (Dsd)',
              'Humid Continental (Dwa)','Humid Continental (Dwb)','Subarctic (Dwc)','Subarctic (Dwd)',
              'Humid Continental (Dfa)','Humid Continental (Dfb)','Subarctic (Dfc)','Subarctic (Dfd)',
              'Tundra (ET)', 'Tundra (EF)']
IGBPList = ['Evergreen Needleleaf Forests', 'Evergreen Broadleaf Forests', 
            'Deciduous Needleleaf Forests', 'Deciduous Broadleaf Forests', 
            'Mixed Forests', 
            'Closed Shrublands', 'Open Shrublands', 
            'Woody Savannas', 'Savannas', 
            'Grasslands', 'Permanent Wetlands', 'Croplands', 
            'Urban and Built-up Lands', 'Cropland/Natural Vegetation Mosaics', 
            'Permanent Snow and Ice', 'Barren', 'Water Bodies']

igbp_dict = {i: IGBPList[i-1] for i in range(1, 18)}
koppen_dict = {i: KoppenList[i-1] for i in range(1, 31)}  

df1['Koppen_Name'] = df1['Koppen_Together'].map(koppen_dict)
df1['IGBP_Name'] = df1['IGBP'].map(igbp_dict)

stats = df1.groupby(['IGBP_Name', 'Koppen_Name']).agg(
    sbedrock_mean=('Sbedrock', 'mean'),    
    sbedrock_median=('Sbedrock', 'median'),
    sbedrock_std=('Sbedrock', 'std'),      
    area_total=('Area', 'sum'),
    # DF = ('Dbedrock_Frequency', 'sum'),             
).reset_index()

df_area = pd.pivot_table(
    df1,
    index=['IGBP_Name', 'Koppen_Name'],
    columns='Dbedrock_Frequency',
    values='Area',
    aggfunc='sum',
    fill_value=0
).reset_index()

df_area.columns = ['IGBP_Name', 'Koppen_Name', 'Pattern1_area', 'Pattern2_area']

stats = stats.merge(df_area, on=['IGBP_Name', 'Koppen_Name'], how='left')

# df_counts = df1.groupby(['IGBP_Name', 'Koppen_Name'])['Dbedrock_Frequency'].value_counts().unstack(fill_value=0).reset_index()
# stats = stats.merge(df_counts, on=['IGBP_Name', 'Koppen_Name'], how='left')

stats['area_total'] = stats['area_total']/1e6
stats['Pattern1_area'] = stats['Pattern1_area'] / 1e6
stats['Pattern2_area'] = stats['Pattern2_area'] / 1e6
stats_sorted = stats.sort_values(by='area_total', ascending=False)
stats_sorted


,IGBP_Name,Koppen_Name,sbedrock_mean,sbedrock_median,sbedrock_std,area_total,DF,Pattern1_area,Pattern2_area
83,Savannas,Tropical Savanna (Aw),225.534062,218.523620,86.916853,2.795885e+06,20265860.0,2.304513e+06,4.913726e+05
85,Woody Savannas,Humid Continental (Dfa),33.909553,21.588599,38.903040,1.546279e+06,46076672.0,3.136161e+05,1.232663e+06
52,Mixed Forests,Humid Continental (Dfa),41.559738,35.114550,35.095100,1.515584e+06,47943923.0,1.287978e+05,1.386786e+06
37,Evergreen Broadleaf Forests,Tropical Monsoon (Am),237.076226,236.150700,111.532650,1.335298e+06,14683414.0,7.391841e+05,5.961137e+05
38,Evergreen Broadleaf Forests,Tropical Rainforest (Af),128.316365,101.339480,111.574232,1.329113e+06,22456795.0,1.740290e+05,1.155084e+06
74,Savannas,Humid Continental (Dfa),27.599767,14.355239,36.457278,1.062129e+06,33049542.0,2.305977e+05,8.315309e+05
39,Evergreen Broadleaf Forests,Tropical Savanna (Aw),243.355728,237.306735,114.684048,1.042775e+06,8257114.0,8.072430e+05,2.355316e+05
62,Open Shrublands,Desert (BWh),32.286091,22.786024,31.205768,9.917964e+05,18734062.0,1.168471e+05,8.749492e+05
78,Savannas,Humid Subtropical (Cwa),181.922751,187.152130,102.639187,8.087397e+05,8810109.0,4.953867e+05,3.133530e+05
63,Open Shrublands,Humid Continental (Dfa),13.040097,7.448474,15.041340,7.428197e+05,25796791.0,2.349165e+05,5.079032e+05


In [ ]:
stats_sorted2 = stats_sorted[stats_sorted['area_total']>4e5]

stats_sorted2['Pattern1'] = (stats_sorted2['Pattern1_area']/stats_sorted2['area_total']*100).round().astype(int)
stats_sorted2['Pattern2'] = (stats_sorted2['Pattern2_area']/stats_sorted2['area_total']*100).round().astype(int)

stats_sorted2['sbedrock_mean'] = stats_sorted2['sbedrock_mean'].round().astype(int)
stats_sorted2['sbedrock_median'] = stats_sorted2['sbedrock_median'].round().astype(int)
stats_sorted2['sbedrock_std'] = stats_sorted2['sbedrock_std'].round().astype(int)
stats_sorted2['area_total'] = stats_sorted2['area_total'].round().astype(int)
stats_sorted2['Pattern1_area'] = stats_sorted2['Pattern1_area'].round().astype(int)
stats_sorted2['Pattern2_area'] = stats_sorted2['Pattern2_area'].round().astype(int)

stats_sorted2

# print(stats_sorted2)

/tmp/ipykernel_839947/2849213252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_sorted2['Pattern1'] = (stats_sorted2['Pattern1_area']/stats_sorted2['area_total']*100).round().astype(int)
/tmp/ipykernel_839947/2849213252.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_sorted2['Pattern2'] = (stats_sorted2['Pattern2_area']/stats_sorted2['area_total']*100).round().astype(int)
/tmp/ipykernel_839947/2849213252.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,IGBP_Name,Koppen_Name,sbedrock_mean,sbedrock_median,sbedrock_std,area_total,DF,Pattern1_area,Pattern2_area,Pattern1,Pattern2
83,Savannas,Tropical Savanna (Aw),226,219,87,2795885,20265860.0,2304513,491373,82,18
85,Woody Savannas,Humid Continental (Dfa),34,22,39,1546279,46076672.0,313616,1232663,20,80
52,Mixed Forests,Humid Continental (Dfa),42,35,35,1515584,47943923.0,128798,1386786,8,92
37,Evergreen Broadleaf Forests,Tropical Monsoon (Am),237,236,112,1335298,14683414.0,739184,596114,55,45
38,Evergreen Broadleaf Forests,Tropical Rainforest (Af),128,101,112,1329113,22456795.0,174029,1155084,13,87
74,Savannas,Humid Continental (Dfa),28,14,36,1062129,33049542.0,230598,831531,22,78
39,Evergreen Broadleaf Forests,Tropical Savanna (Aw),243,237,115,1042775,8257114.0,807243,235532,77,23
62,Open Shrublands,Desert (BWh),32,23,31,991796,18734062.0,116847,874949,12,88
78,Savannas,Humid Subtropical (Cwa),182,187,103,808740,8810109.0,495387,313353,61,39
63,Open Shrublands,Humid Continental (Dfa),13,7,15,742820,25796791.0,234917,507903,32,68


: 